#Ingestion del archivo movie.csv


### Paso 1 - Leer el archivo CSV usando "DataframeReader"

In [0]:
dbutils.widgets.text("p_environment", "")
v_enviroment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType

In [0]:
movie_schema = StructType(fields=[
    StructField("movieId", IntegerType(), False),
    StructField("title", StringType(), True),
    StructField("budget", DoubleType(), True),
    StructField("homepage", StringType(), True),
    StructField("overview", StringType(), True),
    StructField("popularity", DoubleType(), True),
    StructField("yearReleaseDate", IntegerType(), True),
    StructField("releaseDate", DateType(), True),
    StructField("revenue", DoubleType(), True),
    StructField("durationTime", IntegerType(), True),
    StructField("movieStatus", StringType(), True),
    StructField("tagline", StringType(), True),
    StructField("voteAverage", DoubleType(), True),
    StructField("voteCount", IntegerType(), True),
    ])

In [0]:
movie_df = spark.read \
    .option("header", True)\
    .schema(movie_schema)\
    .csv(f"{bronze_folder_path}/{v_file_date}/movie.csv")

## Parte 2 : Seleccionar solo las columnas requeridas

In [0]:
movie_selected_df = movie_df.select("movieId", "title", "budget",  "popularity", "yearReleaseDate", "releaseDate", "revenue", "durationTime", "voteAverage", "voteCount")

In [0]:
from pyspark.sql.functions import col


## Paso 3. Cambiar el nombre de las columnas segun lo requerido

In [0]:
movie_rename_df = movie_selected_df \
    .withColumnRenamed("movieId", "movie_id") \
    .withColumnRenamed("yearReleaseDate", "year_release_date") \
    .withColumnRenamed("releaseDate", "release_date") \
    .withColumnRenamed("durationTime", "duration_time") \
    .withColumnRenamed("voteAverage", "vote_average") \
    .withColumnRenamed("voteCount", "vote_count")
display(movie_rename_df)

    

### Paso 4. Agregar la columna "Ingestion date" al dataframe

In [0]:
from pyspark.sql.functions import current_timestamp, lit
# se importan funciones de fecha del dia de ingestion en el sistema y lit que nos sirve para agragar un valor en una columna ya que si se hace sin esta funcion no sera tomada como objeto



In [0]:
movies_final_df = add_ingestion_date(movie_rename_df) \
                    .withColumn("enviroment", lit(v_enviroment)) \
                    .withColumn("file_date", lit(v_file_date))


### Paso 5. Escribir datos en el datalake en formato "Parquet"

In [0]:
#for item_list in movies_final_df.select("file_date").distinct().collect():
 
 #   if (spark._jsparkSession.catalog().tableExists("movie_silver.movies")):
  #      spark.sql(f"alter table movie_silver.movies DROP IF EXISTS partition(file_date='{item_list.file_date}')")



In [0]:
#overwrite_partition (movies_final_df, "movie_silver", "movies", "file_date")

In [0]:
#from delta.tables import *

#if (spark._jsparkSession.catalog().tableExists("movie_silver.movies")):
    
 #   deltaTable = DeltaTable.forPath(spark, '/mnt/moviehistory7809/silver/movies')
                                               

#    deltaTable.alias('tgt') \
  #  .merge(
#        movies_final_df.alias('src'),
 #       "tgt.movie_id = src.movie_id"
 #   ) \
#    .whenMatchedUpdateAll()\
#    .whenNotMatchedInsert() \
#    .execute()
#else:
#    movies_final_df.write.mode("overwrite").partitionBy("file_date").format("delta").saveAsTable("movie_silver.movies")

###Crear tabla delta con funcion Merge

In [0]:
##DROP TABLE IF EXISTS movie_silver.movies;
#dbutils.fs.rm("/mnt/moviehistory7809/silver/movies", True)

In [0]:
merge_key = 'tgt.movie_id = src.movie_id '
merge_delta_data(movies_final_df,"/mnt/moviehistory7809/silver","movie_silver", "movies", "file_date", merge_key )


In [0]:
%sql
use movie_silver;
show tables

In [0]:
%sql
--use movie_silver;
--show tables;
--show databases;
select file_Date,count(*)
from movie_silver.movies
group by file_Date;

In [0]:
%sql
desc history movie_silver.movies

In [0]:
dbutils.notebook.exit("Ejecutado exitosamente")